In [ ]:
from google.colab import drive
ROOT = '/content/drive'
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import sys
from os.path import join 
repo_dir = '/content/drive/MyDrive/gong-metaphor-detection'

In [ ]:
# pip install requirements (takes a while)
#!cd drive/MyDrive/Repos/gong-metaphor-detection/; pip install -r requirements.txt

In [ ]:
!pip install 'transformers==3.5.1'

     |████████████████████████████████| 1.3 MB 3.7 MB/s 
     |████████████████████████████████| 1.1 MB 36.8 MB/s 
     |████████████████████████████████| 2.9 MB 23.8 MB/s 
     |████████████████████████████████| 895 kB 45.1 MB/s 


In [ ]:
!pip install torch==1.4.0

     |████████████████████████████████| 753.4 MB 6.6 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.


In [ ]:
!pip install scikit-learn

In [ ]:
!cd drive/MyDrive/Repos/gong-metaphor-detection/; bash shell-scripts/run_vua_seq_model.sh


03/12/2022 20:38:07 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
Downloading: 100% 482/482 [00:00<00:00, 840kB/s]
Downloading: 100% 899k/899k [00:00<00:00, 5.23MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 3.13MB/s]
Downloading: 100% 1.43G/1.43G [00:32<00:00, 44.1MB/s]
03/12/2022 20:38:47 - INFO - modeling_roberta_metaphor -   hidden_size: 1024, pos_dim: 8, feature_dim: 128
03/12/2022 20:38:47 - INFO - modeling_roberta_metaphor -   classifier dim: 1032
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForMetaphorDetection: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMetaphorDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification 